# Exercício 2

## Enunciado do problema

Construir uma classe Python que implemente o  EdCDSA a partir do “standard” FIPS186-5
   1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
   2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.
   3. Por aplicação da transformação de Fiat-Shamir construa um protocolo de autenticação de desafio-resposta.


## Descrição do problema

O nosso objetivo é construir uma classe em python que implemente o EdCDSA (Elliptic Curve Digital Signature Algorithm), um algoritmo de assinatura digital baseado em curvas elípticas. Para tal, vamos usar uma das curvas elípticas definidas no standard FIPS186-5, a curva “edwards25519” ou “edwards448”.
Por fim, vamos aplicar a transformação de Fiat-Shamir para construir um protocolo de autenticação de desafio-resposta.

...

## Abordagem

Foi escolhida a curva “edwards25519” para a implementação do EdCDSA. Para a implementação de um DSA (Digital Signature Algorithm) é preciso ter em atenção as seguintes operações:

- Geração de chaves
- Distribuição de chaves
- Assinatura digital
- Verificação da assinatura

De notar que uma curva elíptica só é considerada uma curva de Edwards se satisfizer a seguinte equação:
`a * x^2 + y^2 = 1 + d * x^2 * y^2`

# Código

...

Classe que implementa uma curva elíptica de Edwards

In [1]:
from __future__ import annotations

from sage.all import ZZ
from sage.arith.misc import is_prime
# noinspection PyUnresolvedReferences
from sage.rings.finite_rings.integer_mod import Mod
from sage.schemes.elliptic_curves.constructor import EllipticCurve
from sage.schemes.elliptic_curves.sha_tate import factor


# noinspection PyPep8Naming
class Ed(object):
    def __init__(self, p, a, d, ed=None):
        assert a != d and is_prime(p) and p > 3
        K = GF(p)

        A = 2 * (a + d) / (a - d)
        B = 4 / (a - d)

        alfa = A / (3 * B)
        s = B

        a4 = s ^ (-2) - 3 * alfa ^ 2
        a6 = -alfa ^ 3 - a4 * alfa

        self.K = K
        self.constants = {'a': a, 'd': d, 'A': A, 'B': B, 'alfa': alfa, 's': s, 'a4': a4, 'a6': a6}
        self.EC = EllipticCurve(K, [a4, a6])

        if ed is not None:
            self.L = ed['L']
            self.P = self.ed2ec(ed['Px'], ed['Py'])  # gerador do gru
        else:
            self.gen()

    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h"
        oo = self.EC.order()
        n, _ = list(factor(oo))[-1]
        return n, oo // n

    def gen(self):
        L, h = self.order()
        P = O = self.EC(0)
        while L * P == O:
            P = self.EC.random_element()
        self.P = h * P
        self.L = L

    def is_edwards(self, x, y):
        a = self.constants['a']
        d = self.constants['d']
        x2 = x ^ 2
        y2 = y ^ 2
        return a * x2 + y2 == 1 + d * x2 * y2

    def ed2ec(self, x, y):  ## mapeia Ed --> EC
        if (x, y) == (0, 1):
            return self.EC(0)
        z = (1 + y) / (1 - y)
        w = z / x
        alfa = self.constants['alfa']
        s = self.constants['s']
        return self.EC(z / s + alfa, w / s)

    def ec2ed(self, P):  ## mapeia EC --> Ed
        if P == self.EC(0):
            return 0, 1
        x, y = P.xy()
        alfa = self.constants['alfa']
        s = self.constants['s']
        u = s * (x - alfa)
        v = s * y
        return u / v, (u - 1) / (u + 1)

Classe que implementa operações sobre uma curva elíptica de Edwards

In [50]:


# noinspection PyPep8Naming
class EdPoint(object):
    def __init__(self, pt=None, curve=None, x=None, y=None):
        if pt is not None:
            self.curve = pt.curve
            self.x = pt.x
            self.y = pt.y
            self.w = pt.w
        else:
            assert isinstance(curve, Ed) and curve.is_edwards(x, y)
            self.curve = curve
            self.x = x
            self.y = y
            self.w = x * y

    def eq(self, other):
        return self.x == other.x and self.y == other.y

    def copy(self):
        return EdPoint(curve=self.curve, x=self.x, y=self.y)

    def zero(self):
        return EdPoint(curve=self.curve, x=0, y=1)

    def sim(self):
        return EdPoint(curve=self.curve, x=-self.x, y=self.y)

    def soma(self, other):
        a = self.curve.constants['a']
        d = self.curve.constants['d']
        delta = d * self.w * other.w
        self.x, self.y = (self.x * other.y + self.y * other.x) / (1 + delta), (
                self.y * other.y - a * self.x * other.x) / (1 - delta)
        self.w = self.x * self.y

    def duplica(self):
        a = self.curve.constants['a']
        d = self.curve.constants['d']
        delta = d * self.w ^ 2
        self.x, self.y = (2 * self.w) / (1 + delta), (self.y ^ 2 - a * self.x ^ 2) / (1 - delta)
        self.w = self.x * self.y

    def mult(self, n):
        m = Mod(n, self.curve.L).lift().digits(2)  ## obter a representação binária do argumento "n"
        Q = self.copy()
        A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A

    def encode(self) -> bytes:
        """
        Encode a point in Ed25519 format.
        The input point should be a tuple (x, y) with integers in the range 0 ≤ x, y < p.
        """
        x_int = int(self.x)
        y_int = int(self.y)

        # Copying the least significant bit of the x-coordinate to the most significant bit of the final octet.
        return (int(y_int | ((x_int & 1) << 255))).to_bytes(32, "little")

    @staticmethod
    def decode(s: bytes) -> EdPoint:
        """
        Decode a point in Ed25519 format.
        The output point is a tuple (x, y) with integers in the range 0 ≤ x, y < p.
        """
        assert len(s) == 32
        # 1. Interpret the octet string as an integer in little-endian representation. The most significant bit of this integer is the least significant bit of the x-coordinate, denoted as x0. The y-coordinate is recovered simply by clearing this bit. If the resulting value is ≥ p, decoding fails.
        # TODO: Fazer o 1
        y = int.from_bytes(s, "little") & 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF7F
        x0 = int.from_bytes(s, "little") >> 255
        if not (y < EdCDSA25519.q):
            raise ValueError("Decoding failed")

        # 2. To recover the x-coordinate, the curve equation requires x2 = (y2 - 1) / (d y2 - a) (mod p). The denominator is always non-zero mod p. Compute a square root to obtain x. Square roots can be computed using the Tonelli-Shanks algorithm.
        # TODO: Fazer o 2
        # Simplified cases to compute the square root:
        # Let u = y^2 - 1 and v = d y^2 + 1.
        d = EdCDSA25519.d
        p = EdCDSA25519.q
        u = y ^ 2 - 1
        v = d * y ^ 2 + 1

        # b) To find a square root of (u/v) if p ≡ 5 (mod 8) (as in Ed25519), first compute the candidate root w = (u/v) (p+3)/8 = u v^3 (u v^7)^(p-5)/8 (mod p).
        w = ((u * v ^ 3) * ((u * (v ^ 7)) ^ (p - 5) / 8)) % p
        print("w: ", w)

        # To find the root, check three cases:
        # If v w^2 = u (mod p), the square root is x = w.
        print("v * w ^ 2 == u % p <=> ", v * w ^ 2, " == ", u % p, " <=> ", v * w ^ 2 == u % p)
        print("v * w ^ 2 == -u % p <=> ", v * w ^ 2, " == ", -u % p, " <=> ", v * w ^ 2 == -u % p)
        if v * w ^ 2 == u % p:
            x = w
        # If v w^2 = -u (mod p), the square root is x = w * 2^((p-1)/4).
        elif v * w ^ 2 == -u % p:
            x = w * 2 ^ ((p - 1) / 4)
        # Otherwise, no square root exists for modulo p, and decoding fails.
        else:
            raise ValueError("No square root exists for modulo p, and decoding fails.")

        # For both cases, if x = 0 and x0 = 1, point decoding fails.
        if x == 0 and x0 == 1:
            raise ValueError("Point decoding failed")

        # If x (mod 2) = x0, then the x-coordinate is x.
        if x % 2 == x0:
            x = x  # Just to make it explicit
        # Otherwise, the x-coordinate is p - x.
        else:
            x = p - x

        # 3. Return the decoded point (x,y).
        return EdPoint(curve=EdCDSA25519.E, x=x, y=y)


Classe que implementa a curva de Edwards 25519

In [77]:
import os
from hashlib import sha512
from sage.rings.finite_rings.all import GF


# noinspection PyPep8Naming
class EdCDSA25519:
    q = 2 ** 255 - 19
    K = GF(q)
    a = K(-1)
    d = -K(121665) / K(121666)

    ed25519 = {
        'b': 256,  ## tamanho da chave em bits
        'Px': K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
        # coordenada x do gerador
        'Py': K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
        # coordenada y do gerador
        'L': ZZ(2 ^ 252 + 27742317777372353535851937790883648493),  ## ordem do subgrupo primo
        'n': 254,
        'h': 8,  # cofator do subgrupo
        'c': 3  # logaritmo base 2 do cofator [RFC7748]
    }

    E = Ed(q, a, d, ed25519)  # Curva de Edwards

    def generate_keys(self) -> (bytes, bytes, EdPoint):
        #  1. Obtain a string of b bits from an approved RBG with a security strength of requested_security_strength or more. The private key d is this string of b bits.
        d = os.urandom(self.ed25519[
                           'b'] // 8)  # TODO: Como é que gero uma chave privada de 256 bits? Posso passar isto numa XOF acho
        d = bytes.fromhex("1de6c4c405fd42d3d6981bcf89dd10bade4e43537fa1c7050af28ec2b3b98f95")

        # 2. Compute the hash of the private key d, H(d) = (h0, h1, ..., h2b-1) using SHA-512 for Ed25519. H(d) may be pre-computed. Note H(d) is also used in the EdDSA signature generation;
        private_key_hashed = self.get_hash(d)

        # 3. The first half of H(d), (i.e. hdigest1 = (ℎ0,ℎ1,…,ℎ𝑏𝑏−1)) is used to generate the public key. Modify hdigest1 as follows:
        # 3.1 For Ed25519, the first three bits of the first octet are set to zero; the last bit of the last octet is set to zero; and the second to last bit of the last octet is set to one. That is, ℎ0=ℎ1=ℎ2=0, ℎ𝑏𝑏−2=1, and ℎ𝑏𝑏−1=0.
        hdigest1 = private_key_hashed[:32]

        hdigest1_int0 = int(hdigest1[0])
        hdigest1_int0 &= 31  # h0 = h1 = h2 = 0
        print("hdigest1[0]=", bin(int(hdigest1[0])))
        print("hdigest1_int0=", bin(hdigest1_int0))


        hdigest1_int31 = int(hdigest1[31])
        hdigest1_int31 |= 2  # hbb-2 = 1
        hdigest1_int31 &= 254  # hbb-1 = 0
        print("hdigest1[31]=", bin(int(hdigest1[31])))
        print("hdigest1_int0=", bin(hdigest1_int31))

        hdigest1 = (int(hdigest1_int0)).to_bytes(1, "big") + hdigest1[1:31] + (int(hdigest1_int31)).to_bytes(1, "big")
        # TODO: Falta fazer o 3.1
        print("hdigest1=", bin(int.from_bytes(hdigest1, "big")))

        # 4. Determine an integer s from hdigest1 using little-endian convention (see Section 7.2).
        s = int.from_bytes(hdigest1, "little")
        print("s=", bin(s))

        # 5. Compute the point [s]G. The corresponding EdDSA public key Q is the encoding (See Section 7.2) of the point [s]G.
        P = EdPoint(curve=self.E, x=self.ed25519['Px'], y=self.ed25519['Py'])
        Q = P.mult(s)

        # Encoding the public key
        Q_encoded = Q.encode()

        print("Q_encoded=", Q_encoded.hex())
        #assert Q_encoded == bytes.fromhex("1972E03EDC718B87CC6E141B1C745E115CE8895C96CBF1037DA8EA2E4C8CCE92")

        return d, Q, Q_encoded

    def sign(self, message: bytes, private_key: bytes, public_key: EdPoint, public_key_encoded: bytes) -> bytes:
        # 1. Compute the hash of the private key d, H(d) = (h0, h1, ..., h2b-1) using SHA-512 for Ed25519. H(d) may be pre-computed.
        priv_key_hashed = self.get_hash(private_key)

        # 2. Using the second half of the digest hdigest2 = hb || ... || h2b-1, define:
        # 2.1 For Ed25519, r = SHA-512(hdigest2 || M); r will be 64-octets.
        hdigest2 = priv_key_hashed[32:]
        r = self.get_hash(hdigest2 + message)
        assert len(r) == 64  # r tem de ter 64 octetos

        r_int = int.from_bytes(r, "little")

        # 3. Compute the point [r]G. The octet string R is the encoding of the point [r]G.
        G = EdPoint(curve=self.E, x=self.ed25519['Px'], y=self.ed25519['Py'])  # Generator
        R = G.mult(r_int)

        # 4. Derive s from H(d) as in the key pair generation algorithm. Use octet strings R, Q, and M to define:
        # 4.1 For Ed25519, S = (r + SHA-512(R || Q || M) * s) mod n. -> S = (r + SHA-512(R || public_key || M) * s) mod n.
        # The octet string S is the encoding of the resultant integer.
        R = R.encode()
        Q = public_key_encoded
        M = message
        h = self.get_hash(R + Q + M)
        h_int = int.from_bytes(h, "little")
        s = priv_key_hashed[:32]  # s = hdigest1
        s_int = int.from_bytes(s, "little")

        S = (r_int + h_int * s_int) % self.ed25519['L']

        S_bytes = int(S).to_bytes(32, "little")

        # 5. Form the signature as the concatenation of the octet strings R and S.
        signature = R + S_bytes

        return signature

    def verify(self, message: bytes, signature: bytes, public_key: bytes) -> bool:
        # 1. Decode the first half of the signature as a point R and the second half of the signature as an integer s. Verify that the integer s is in the range of 0 ≤ s < n. Decode the public key Q into a point Q’. If any of the decodings fail, output “reject”.
        # TODO: Fazer o passo 1
        R = signature[:32]
        s = int.from_bytes(signature[32:], "little")
        assert 0 <= s < self.ed25519['L']

        # 2. Form the bit string HashData as the concatenation of the octet strings R, Q, and M (i.e., HashData = R || Q || M).
        # TODO: Fazer o passo 2
        Q = public_key
        M = message
        HashData = R + Q + M

        # 3. Using the established hash function or XOF,
        # 3.1 For Ed25519, compute digest = SHA-512(HashData).
        # Interpret digest as a little-endian integer t.
        # TODO: Fazer o passo 3.1
        digest = self.get_hash(HashData)
        t = int.from_bytes(digest, "little")

        # 4. Check that the verification equation [2^c * S]G = [2^c]R + (2^c * t)Q. Output “reject” if verification fails; output “accept” otherwise.
        # TODO: Fazer o passo 4
        G = EdPoint(curve=self.E, x=self.ed25519['Px'], y=self.ed25519['Py'])  # Generator
        R = EdPoint.decode(R)
        Q = EdPoint.decode(Q)
        h = self.ed25519['h']  # 2^c = h = 8 (Ed25519)

        return R.mult(h * s) == G.mult(h * t) + R.mult(h * t) * Q

    @staticmethod
    def get_hash(message):
        h = sha512(message).digest()
        return h

# Testes e exemplos

...

In [81]:
E = EdCDSA25519()

#priv_key, pub_key, pub_key_encoded = E.generate_keys()

#print(priv_key.hex())
#print(pub_key_encoded.hex())

pub_key_example = bytes.fromhex("1972E03EDC718B87CC6E141B1C745E115CE8895C96CBF1037DA8EA2E4C8CCE92")

pub_key = EdPoint.decode(pub_key_example)
#assert EdPoint.decode(pub_key_example) == pub_key

w:  47526803810662887076519524208253928251370061005589212062870908476371309312098
v * w ^ 2 == u % p <=>  22202319023128508926243794844375915470646911393615189234954252378750999981214  ==  39909771679907184035825657837160013026273556811396117353463816626174756637435  <=>  False
v * w ^ 2 == -u % p <=>  22202319023128508926243794844375915470646911393615189234954252378750999981214  ==  17986272938750913675959834667183940900361435521424164666264975377781808182514  <=>  False


ValueError: No square root exists for modulo p, and decoding fails.

## Exemplo 1

In [62]:
E = EdCDSA25519()

priv_key, pub_key, pub_key_encoded = E.generate_keys()

print("Private Key:", priv_key.hex())
print(f"Public Key: {pub_key_encoded.hex()}")

Private Key: b72647493fe179da264ee6f98c4dc59365ad3a2f882f077cc65b5323399a2e72
Public Key: a8ec6c0cd6c71a62b3cc8737ac8b7cc17fe9c9fe6a39dfa9cbfd44e30aaa0bec


In [54]:
m1 = bytes("Hello World", "utf-8")
m2 = os.urandom(32)

m2_signature = E.sign(m2, priv_key, pub_key, pub_key_encoded)

print("Message:", m2)
print("Signature:", m2_signature)

Message: b'\xa3\x1eT\x9d&=\x08a\xd7^\xd5\x1a\xef;I\x95\xf0\x94\x9d}\xdb\x8de\x03\xbd\xedP\xc4\xbc\xe1\xf0\x02'
Signature: b'\xd0\xa3\xea\xaf\xf07yM4\xe6\xb8\xc2\xb1\xd7\x9c\xa0\x85\xd3\x10Q\xe2_.!K\xe0]uq\xa5\x10\xaat\xc2\x0ei\x15\xf9\xca\xff\xcb\xa3G\xbf\xa6l\x8dC\x05\xbaw\x9b\xa4Bz\xb3\xdb\xae\xee\xdc\r\x1a>\x0c'


In [58]:
print(priv_key.hex())
print(pub_key_encoded.hex())

b72647493fe179da264ee6f98c4dc59365ad3a2f882f077cc65b5323399a2e72
a8ec6c0cd6c71a62b3cc8737ac8b7cc17fe9c9fe6a39dfa9cbfd44e30aaa0bec
